# Microsoft Orleans Hello World Example
This notebook shows how to write a simple [Microsoft Orleans](https://github.com/dotnet/orleans) client &lt;=&gt; server application.

Since .NET Interactive Notebook is not a pure C# project, we cannot use the Build time code generation packages but instead install [`Microsoft.Orleans.OrleansCodeGenerator`]() that do code generation when both client & silo app starting:

In [ ]:
#region install Nuget

//Essential libs for running .NET 6 Generic Host
#r "nuget: Microsoft.Extensions.Hosting"
#r "nuget: Microsoft.Extensions.DependencyInjection"

//Logging libraries
#r "nuget: Microsoft.Extensions.Logging.Abstractions,6.0.0"
#r "nuget: Microsoft.Extensions.Logging,6.0.0"
#r "nuget: Serilog.Extensions.Logging"
#r "nuget: Serilog.Sinks.Console"

//Orleans essential dependencies
#r "nuget: Microsoft.Orleans.Core"
#r "nuget: Microsoft.Orleans.OrleansRuntime"
//Orleans RPC method code generation library
#r "nuget: Microsoft.Orleans.OrleansCodeGenerator"

//Orleans seperate silo server runtime essentials
#r "nuget: Microsoft.Orleans.Server"

#endregion

Installed Packages Microsoft.Extensions.DependencyInjection, 6.0.0 Microsoft.Extensions.Hosting, 6.0.1 Microsoft.Extensions.Logging, 6.0.0 Microsoft.Extensions.Logging.Abstractions, 6.0.0 Microsoft.Orleans.Core, 3.6.2 Microsoft.Orleans.OrleansCodeGenerator, 3.6.2 Microsoft.Orleans.OrleansRuntime, 3.6.2 Microsoft.Orleans.Server, 3.6.2 Serilog.Extensions.Logging, 3.1.0 Serilog.Sinks.Console, 4.0.1

In [ ]:
using Orleans;
using Orleans.Runtime;

using Microsoft.Extensions.Hosting;
using Orleans.Hosting;

using Microsoft.Extensions.Logging;
using Serilog;

## Grain Development
Grain is the essential actor part of Orleans. It can be called on Client-side and also being invoked on Server-side that used to communicate with other grain(s).
### Grain Interface
Grain's interface is the essential part of Orleans RPC method declaration, which use [TPL](https://docs.microsoft.com/en-us/dotnet/standard/parallel-programming/task-parallel-library-tpl)'s [`Task<TResult>`](https://docs.microsoft.com/en-us/dotnet/api/system.threading.tasks.task-1) to return the RPC result.


In [ ]:
#!pwsh
cat ./class_libs/HelloWorld.Interfaces/IHelloGrain.cs

using Orleans;

namespace HelloWorld.Interfaces;
public interface IHelloGrain : IGrainWithStringKey
{
    Task<string> SayHello(string greeting);
}


### Grain Class
The actual virtual actor class code that implements Business logic, currently it must inherit from [`Grain`](https://docs.microsoft.com/en-us/dotnet/api/orleans.grain) class.
The **POCO(Plane Old CLR Objects)** way to develop grain is scheduled to [be available on Orleans v4](https://github.com/dotnet/orleans/issues/7351).

In [ ]:
#!pwsh
cat ./class_libs/HelloWorld.Grains/HelloGrain.cs

using Orleans;
using Orleans.Runtime;

using HelloWorld.Interfaces;

namespace HelloWorld.Grains;
public class HelloGrain : Grain, IHelloGrain
{
    public Task<string> SayHello(string greeting) => Task.FromResult($"Hello, {greeting}!");
}


Because Notebook use C# Scripting which cannot declare namespace, and the Orleans's code generator has bug on global level type: https://github.com/dotnet/orleans/issues/6214, we could only use project code by build it into actual assembly files.

In [ ]:
#!pwsh
dotnet build ./class_libs/HelloWorld.Grains/HelloWorld.Grains.csproj --nologo --verbosity minimal

  Determining projects to restore...
  Restored D:\myWorkDir\jupyter_notebooks\Orleans_example\class_libs\HelloWorld.Grains\HelloWorld.Grains.csproj (in 175 ms).
  Restored D:\myWorkDir\jupyter_notebooks\Orleans_example\class_libs\HelloWorld.Interfaces\HelloWorld.Interfaces.csproj (in 175 ms).
  Orleans.CodeGenerator - command-line = SourceToSource D:\myWorkDir\jupyter_notebooks\Orleans_example\class_libs\HelloWorld.Interfaces\obj\Debug\net6.0\HelloWorld.Interfaces.orleans.g.args.txt
  HelloWorld.Interfaces -> D:\myWorkDir\jupyter_notebooks\Orleans_example\class_libs\HelloWorld.Interfaces\bin\Debug\net6.0\HelloWorld.Interfaces.dll
  Orleans.CodeGenerator - command-line = SourceToSource D:\myWorkDir\jupyter_notebooks\Orleans_example\class_libs\HelloWorld.Grains\obj\Debug\net6.0\HelloWorld.Grains.orleans.g.args.txt
  HelloWorld.Grains -> D:\myWorkDir\jupyter_notebooks\Orleans_example\class_libs\HelloWorld.Grains\bin\Debug\net6.0\HelloWorld.Grains.dll

Build succeeded.
    0 Warning(s)
  

Load built assembly files:

In [ ]:
#r "./class_libs/HelloWorld.Interfaces/bin/Debug/net6.0/HelloWorld.Interfaces.dll"
#r "./class_libs/HelloWorld.Grains/bin/Debug/net6.0/HelloWorld.Grains.dll"

## Silo Host Configuration

Create [HostBuilder](https://docs.microsoft.com/en-us/dotnet/api/microsoft.extensions.hosting.hostbuilder) and configure it, we use local development host by calling [`UseLocalhostClustering()`](https://docs.microsoft.com/en-us/dotnet/api/orleans.hosting.corehostingextensions.uselocalhostclustering) extension method:


In [ ]:
using HelloWorld.Grains;
using HelloWorld.Interfaces;

// Optional, the logger factory for seeing the Orleans RPC method stud code gen log.
Log.Logger = new LoggerConfiguration().WriteTo.Console().CreateLogger();
var codeGenLoggerFactory = LoggerFactory.Create(logBuilder =>logBuilder.AddSerilog());

var hostBuilder = new HostBuilder().UseOrleans(siloBuilder =>{
    // config to use development silo 
    siloBuilder.UseLocalhostClustering();

    // Grain and its interface needs to be registered via adding the "ApplicationPart", and also assign the RPC method code generation strategy during host startup.
    siloBuilder.ConfigureApplicationParts(parts =>{
        parts.AddApplicationPart(typeof(HelloGrain).Assembly).WithCodeGeneration(loggerFactory: codeGenLoggerFactory);
        parts.AddApplicationPart(typeof(IHelloGrain).Assembly).WithCodeGeneration(loggerFactory: codeGenLoggerFactory);
    });
});

Build silo host, it will code gen Orleans RPC method:

In [ ]:
var silo_host = hostBuilder.Build();

[17:34:21 INF] Generating code for assemblies: 
[17:34:21 INF] Runtime code generation for assemblies  HelloWorld.Grains, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 459 milliseconds
[17:34:21 INF] Generating code for assemblies: 
[17:34:22 INF] Runtime code generation for assemblies  HelloWorld.Interfaces, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 158 milliseconds


Start silo host by calling [`StartAsync()`](https://docs.microsoft.com/en-us/dotnet/api/microsoft.extensions.hosting.ihost.startasync) extension method:

In [ ]:
await silo_host.StartAsync();

Now we can use .NET Generic Host's dependency injection feature to get a [IGrainFactory](https://docs.microsoft.com/en-us/dotnet/api/orleans.igrainfactory) instance, than use various [`GetGrain()`](https://docs.microsoft.com/en-us/dotnet/api/orleans.igrainfactory.getgrain) overload methods to get a specified Grain reference, then we can invoke RPC method in host-side:

In [ ]:
using Microsoft.Extensions.DependencyInjection;
var grainFactory = silo_host.Services.GetRequiredService<IGrainFactory>();

var helloGrain = grainFactory.GetGrain<IHelloGrain>("demo");
display(await helloGrain.SayHello("Microsoft Orleans"));

Hello, Microsoft Orleans!

## Client Configuration
Create [ClientBuilder](https://docs.microsoft.com/en-us/dotnet/api/orleans.clientbuilder) and configure it, we only need to code gen Orleans Interface part:

In [ ]:
var clientBuilder = new ClientBuilder().UseLocalhostClustering();
clientBuilder.ConfigureApplicationParts(parts=>parts.AddApplicationPart(typeof(IHelloGrain).Assembly).WithCodeGeneration(loggerFactory: codeGenLoggerFactory));

[17:34:56 INF] Generating code for assemblies: 
[17:34:56 INF] Runtime code generation for assemblies  HelloWorld.Interfaces, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 134 milliseconds


Get `IClusterClient` orleans client instance by calling [`Build()`](https://docs.microsoft.com/en-us/dotnet/api/orleans.clientbuilder.build) method of clientBuilder, and call [`Connect()`](https://docs.microsoft.com/en-us/dotnet/api/orleans.iclusterclient.connect) method to connect to silo host:

In [ ]:
var orleans_client = clientBuilder.Build();
await orleans_client.Connect();

We can get Grain RPC method stub reference by calling use various [`GetGrain()`](https://docs.microsoft.com/en-us/dotnet/api/orleans.igrainfactory.getgrain) overload methods that  IClusterClient instance inherit from IGrainFactory, and invoke RPC method in client-side:

In [ ]:
var clientSideHelloGrain = orleans_client.GetGrain<IHelloGrain>("demo");
display(await clientSideHelloGrain.SayHello("invoke Microsoft Orleans RPC method from client"));

Hello, invoke Microsoft Orleans RPC method from client!

Remember to call [`Close()`](https://docs.microsoft.com/en-us/dotnet/api/orleans.iclusterclient.close) when client-side don't need to use anymore.

In [ ]:
await orleans_client.Close();

Shutdown silo host by calling [`StopAsync()`](https://docs.microsoft.com/en-us/dotnet/api/microsoft.extensions.hosting.hostingabstractionshostextensions.stopasync) extension method:

In [ ]:
await silo_host.StopAsync();